## Geographically Weighted Correlation Coefficient

### Import relevant dependencies

In [1]:
import pandas as pd
import numpy as np
import geopandas as gp
import libpysal as ps
from mgwr.gwr import GWR, MGWR, GWRResults
from mgwr.sel_bw import Sel_BW

In [2]:
georgia_data = pd.read_csv(ps.examples.get_path('GData_utm.csv'))
georgia_shp = gp.read_file(ps.examples.get_path('G_utm.shp'))

#### Prepare the Georgia dataset inputs

In [3]:
g_y = georgia_data['PctBach'].values.reshape((-1,1))
g_X = georgia_data[['PctFB']].values
u = georgia_data['X']
v = georgia_data['Y']


g_coords = list(zip(u,v))

In [4]:
georgia_data.head()

,AreaKey,Latitude,Longitud,TotPop90,PctRural,PctBach,PctEld,PctFB,PctPov,PctBlack,ID,X,Y
0,13001,31.75339,-82.28558,15744,75.6,8.2,11.43,0.64,19.9,20.76,133,941396.6,3521764.0
1,13003,31.29486,-82.87474,6213,100.0,6.4,11.77,1.58,26.0,26.86,158,895553.0,3471916.0
2,13005,31.55678,-82.45115,9566,61.7,6.6,11.11,0.27,24.1,15.42,146,930946.4,3502787.0
3,13007,31.33084,-84.45401,3615,100.0,9.4,13.17,0.11,24.8,51.67,155,745398.6,3474765.0
4,13009,33.07193,-83.25085,39530,42.7,13.3,8.64,1.43,17.5,42.39,79,849431.3,3665553.0


#### Standardization Routine

In [5]:
# g_X = (g_X - g_X.mean(axis=0)) / g_X.std(axis=0)

# g_y = g_y.reshape((-1,1))  

# g_y = (g_y - g_y.mean(axis=0)) / g_y.std(axis=0)

#### Calibrate GWR model

In [6]:
g_y = georgia_data['PctBach'].values.reshape((-1,1))
g_X = georgia_data[['PctFB']].values
u = georgia_data['X']
v = georgia_data['Y']


g_coords = list(zip(u,v))


gwr_selector = Sel_BW(g_coords, g_y, g_X, constant=True)
gwr_bw = gwr_selector.search()

gwr_results = GWR(g_coords, g_y, g_X, 50, constant=True).fit()

In [7]:
gwr_results.summary()

Model type                                                         Gaussian
Number of observations:                                                 159
Number of covariates:                                                     2

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                             87.210
Log-likelihood:                                                    -177.864
AIC:                                                                359.729
AICc:                                                               361.883
BIC:                                                               -708.608
R2:                                                                   0.452
Adj. R2:                                                              0.448

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

### GW - Correlation Coefficient


$$
   \frac{(x_j - \overline x_i)(y_j - \overline y_i) w_ij}{\sqrt w_ij(x_j - \overline x_i)^2 . \sqrt w_ij(y_j - \overline y_i)^2 }\
$$

From Page 162-163 of [GWR - The analysis of spatially varying relationships](https://www.academia.edu/33626785/Geographically_Weighted_Regression_The_Analysis_of_Spatially_Varying_Relationships_Wiley_2002)


#### Implementation

In [18]:
# wi =  gwr_results.model._build_wi(0, 50).reshape(-1,1)
# x = g_X
# xi_mean = (x*wi).mean(axis=0)
# y= g_y
# yi_mean = (y*wi).mean(axis=0)

# xi = x*wi
# yi = y*wi



corr = []

for i in range(len(g_X)):
    wi = gwr_results.model._build_wi(i, 50).reshape(-1,1)
    x = g_X
    xi_mean = (x*wi).mean(axis=0)
    y= g_y
    yi_mean = (y*wi).mean(axis=0)
    numerator = wi*((x - xi_mean) * (y - yi_mean))
    denom = ((wi*(np.sqrt((x - xi_mean)**2))) * (wi*(np.sqrt((y - yi_mean)**2))))
    r = (numerator/denom)
#     r[np.isnan(r)] = 0
#     corr.append(r)

# np.array(corr).mean()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_28524\4103135464.py:22: RuntimeWarning: invalid value encountered in true_divide
  r = (numerator/denom)


In [9]:
# !pip install statsmodels

In [11]:
import statsmodels.api as sm

wi =  gwr_results.model._build_wi(0, 50).reshape(-1,1)

y = georgia_data['PctBach'].values.reshape((-1,1))
x = georgia_data[['PctFB']].values


wi = np.sqrt(wi)

xi = x*wi
yi = y*wi

corr_coef = np.corrcoef(xi.flatten(), yi.flatten())
print(corr_coef)

# xi - xi.mean(axis=0) / xi.std(axis=0)
# yi - yi.mean(axis=0) / yi.std(axis=0)



sm.OLS(yi, xi).fit().summary()

[[1.         0.72793246]
 [0.72793246 1.        ]]


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.599
Model:                            OLS   Adj. R-squared (uncentered):              0.596
Method:                 Least Squares   F-statistic:                              235.6
Date:                Wed, 21 Sep 2022   Prob (F-statistic):                    3.93e-33
Time:                        14:27:21   Log-Likelihood:                         -341.92
No. Observations:                 159   AIC:                                      685.8
Df Residuals:                     158   BIC:                                      688.9
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             6.1545      0.401     15.350      0.000       5.363       6.946
==============================================================================
Omnibus:                       29.978   Durbin-Watson:                   1.865
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              116.324
Skew:                           0.590   Prob(JB):                     5.50e-26
Kurtosis:                       7.021   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [33]:
# gwr_results.model._build_wi(0, 116)

In [7]:
numerator = (wi*(x - xi) * (y - yi))
denom = (np.sqrt((x - xi)**2)) * (np.sqrt((y - yi)**2))

In [8]:
r = (numerator/denom) 

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9540\699091297.py:1: RuntimeWarning: invalid value encountered in true_divide
  r = (numerator/denom)


In [9]:
r.mean(axis=0)  # correlation coefficient for each of the variables

array([nan])

In [10]:
gwr_results.params[0,1]

1.1823563464824765

The results in the above cell shows that if one weights the data and not standardize it right after, GWR slope/coefficients != GW correlation coefficient


But I am not able to show otherwise yet i.e standardize after weighting, because of the blockers I am having with the data formats. Which I am currently working to fix. My code is in the last cell. 

#### Global correlation coefficients

In [8]:
from scipy.stats import pearsonr

for i in range(3):
    corr, u = pearsonr(g_X[:,i], g_y.flatten())
    
    print(corr)

0.6719466884233966


IndexError: index 1 is out of bounds for axis 1 with size 1

 The cell below has my implementation of the _compute_betas_gwr()_ function from inside of the conda environment `spglm/iwls.py` source code. 

In [ ]:
def _compute_betas_gwr(y, x, wi):
    """
    compute MLE coefficients using iwls routine

    Methods: p189, Iteratively (Re)weighted Least Squares (IWLS),
    Fotheringham, A. S., Brunsdon, C., & Charlton, M. (2002).
    Geographically weighted regression: the analysis of spatially varying relationships.
    """

    xw = (x * wi).T  # weight before standardization -> result is the weighted design matrix

    xw_stdz = scaler.fit_transform(xw)   # standardize the design matrix -- after weighting
    
    x_stdz = scaler.fit_transform(x)     # standardize x 
                    
    y = scaler.fit_transform(y)          # standardize the y


    xtx = np.dot(xw_stdz, x_stdz)
    xtx_inv_xt = linalg.solve(xtx, xw_stdz)
    betas = np.dot(xtx_inv_xt, y)
    return betas, xtx_inv_xt